# Simple Trapezoid Fit Example

This is basically the unit test modified for an interactive session in a notebook format, so you can play with the API.

A development version of `exovetter` is required to be installed, along with all the required and optional dependencies. Refer to package documentation for more information.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.utils.compat.context import nullcontext
from astropy.utils.misc import NumpyRNGContext

from exovetter.trapezoid_fit import TrapezoidFitParameters, TrapezoidOriginalEstimates, TrapezoidFit, phase_data

%matplotlib inline

Set the random seed for reproducibility. Set it to `None` if you want a truly random behavior.

In [ ]:
seed = 1234

In [ ]:
if seed is None:
    rand_ctx = nullcontext()
else:
    rand_ctx = NumpyRNGContext(seed)

Adjust the values here according to your science case. Please be mindful of the units.

In [ ]:
data_span = 80.0  # in Days
exposure_length = 1.0 / 48.0  # in Days, 48 cadences per day
signal_depth = 300.0  # signal depth in ppm
signal_duration = 5.0 / 24.0  # in Days
signal_period = 10.4203  # in Days
signal_epoch = 5.1  # in Days
noise_level = 40.0  # noise per observation in ppm
samplen = 15
fitregion = 4.0

These are computed parameters based on your inputs above.

In [ ]:
n_data = int(data_span / exposure_length)
signal_duration_hours = signal_duration * 24.0

time_series = np.linspace(0, int(data_span), n_data)

with rand_ctx:
    data_series = 1.0 + np.random.randn(n_data) / 1e6 * noise_level

error_series = np.full_like(data_series, noise_level / 1e6)

Pass the parameters into classes that hold fit parameters and original estimates, respectively.

In [ ]:
trp_parm = TrapezoidFitParameters(exposure_length, samplen=samplen, fitregion=fitregion)
trp_origests = TrapezoidOriginalEstimates(
    period=signal_period, epoch=signal_epoch,
    duration=signal_duration_hours, depth=signal_depth)

Instantiate the class to do the trapezoid fitting.

In [ ]:
ioblk = TrapezoidFit(
    time_series, data_series, error_series,
    trp_parameters=trp_parm, trp_originalestimates=trp_origests,
    t_ratio=0.1)

Make a model trapezoid light curve.

In [ ]:
ioblk.trapezoid_model()

Insert signal.

In [ ]:
data_series *= ioblk.modellc

Some values need to be readjusted before fitting.

In [ ]:
new_signal_epoch = signal_epoch + 0.001
new_signal_duration_hours = signal_duration_hours * 0.9
new_signal_depth = signal_depth * 1.1
fit_trial_n = 2
error_scale = 1.0

Perform the fitting.

In [ ]:
ioblk = TrapezoidFit.trapezoid_fit(
    time_series, data_series, error_series,
    signal_period, new_signal_epoch,
    new_signal_duration_hours, new_signal_depth,
    fit_trial_n=fit_trial_n, fit_region=fitregion,
    error_scale=error_scale, sample_n=samplen, seed=seed)

Print the planet model estimates.

In [ ]:
print(ioblk.planetests)

Plot the fitting result.

In [ ]:
ioblk.plot_likehood()

More plots.

In [ ]:
phased_series = phase_data(time_series, signal_period, signal_epoch)

In [ ]:
plt.plot(phased_series, data_series, '.');

In [ ]:
plt.plot(time_series, data_series, '.');

## Encore: Other Ways to Generate Models

This is the other unit test for alternative ways to general models without doing the fitting part.

In [ ]:
newioblk = TrapezoidFit.trapezoid_model_onemodel(
    time_series, signal_period,
    signal_epoch, signal_depth, signal_duration_hours,
    signal_duration_hours * 0.1, samplen)

In [ ]:
newioblk2 = newioblk.trapezoid_model_raw(
    signal_epoch + 0.05, signal_depth * 1.5,
    signal_duration_hours * 2.0,
    signal_duration_hours * 2.0 * 0.2)

In [ ]:
fig = plt.figure()
ax = fig.subplots()
ax.plot(phased_series, newioblk.modellc, '.b')
ax.plot(phased_series, newioblk2.modellc, '.r')
ax.set_xlim(-0.05, 0.05);